In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from weibull_model_football_scores.probs.weibull import weibull_likelihood

In [69]:
import pandas as pd
from weibull_model_football_scores.ingestion import load_data, load_prediction_dataset
from weibull_model_football_scores.model import WeibullCountFitter

from sklearn.metrics import precision_recall_fscore_support

In [51]:
df = load_data(league="seriea", season="22_23")

In [52]:
train = df[df["fixture"] < df["fixture"].max() - 3].copy()
test = df[df["fixture"] >= df["fixture"].max() - 3].copy()

In [54]:
w = WeibullCountFitter()

In [55]:
w.fit(train, learning_rate=0.01)

Starting likelihood: -749.0820457577335
Step 1, current likelihood: -708.0310183136045
Step 2, current likelihood: -687.2584045383283
Step 3, current likelihood: -670.2999617323087
Step 4, current likelihood: -656.4138969315983
Step 5, current likelihood: -645.1289538251566
Step 6, current likelihood: -636.0786658062285
Step 7, current likelihood: -628.9445536281493
Step 8, current likelihood: -623.4260717231252
Step 9, current likelihood: -619.2275118452685
Step 10, current likelihood: -616.0649505117524
Step 11, current likelihood: -613.6896843109588
Step 12, current likelihood: -611.905781077931
Step 13, current likelihood: -610.5676074693938
Step 14, current likelihood: -609.5672548145427
Step 15, current likelihood: -608.82348427789
Step 16, current likelihood: -608.2743430452693
Step 17, current likelihood: -607.872139747706
Step 18, current likelihood: -607.5799187556867
Step 19, current likelihood: -607.3690352445088
Step 20, current likelihood: -607.2174820728219
Step 21, curr

In [56]:
w.C_team.sort_values("alpha", ascending=False)

team     alpha      beta
8        Napoli  1.680320  0.035298
10        Inter  1.122591  0.460647
7      Atalanta  1.115084  0.465301
2         Milan  0.973875  0.729060
3         Lazio  0.848351  0.312503
4      Juventus  0.759659 -0.110950
11      Udinese  0.729062  0.499002
12         Roma  0.625701  0.015697
6       Bologna  0.589335  0.811937
15        Monza  0.522905  0.762970
16  Salernitana  0.521193  1.163478
5         Lecce  0.405732  0.461894
9        Torino  0.380563  0.489980
0      Sassuolo  0.340906  0.865614
18       Empoli  0.236458  0.618185
17   Fiorentina  0.212121  0.647365
1        Verona  0.155646  0.830846
19    Cremonese  0.080436  1.127844
13       Spezia -0.038302  1.034500
14    Sampdoria -0.362600  0.971094

In [57]:
w.C_team.sort_values("beta", ascending=True)

team     alpha      beta
4      Juventus  0.759659 -0.110950
12         Roma  0.625701  0.015697
8        Napoli  1.680320  0.035298
3         Lazio  0.848351  0.312503
10        Inter  1.122591  0.460647
5         Lecce  0.405732  0.461894
7      Atalanta  1.115084  0.465301
9        Torino  0.380563  0.489980
11      Udinese  0.729062  0.499002
18       Empoli  0.236458  0.618185
17   Fiorentina  0.212121  0.647365
2         Milan  0.973875  0.729060
15        Monza  0.522905  0.762970
6       Bologna  0.589335  0.811937
1        Verona  0.155646  0.830846
0      Sassuolo  0.340906  0.865614
14    Sampdoria -0.362600  0.971094
13       Spezia -0.038302  1.034500
19    Cremonese  0.080436  1.127844
16  Salernitana  0.521193  1.163478

In [63]:
T = test.join(w.predict_under_over(test))

In [66]:
T["over_true"] = T["FTHG"] + T["FTAG"] > 2.5
T["over_pred"] = T["over"] > T["under"] 

In [67]:
T

fixture     HomeTeam     AwayTeam  FTHG  FTAG     under      over  \
230       23       Empoli       Napoli     0     2  0.444633  0.555367   
231       23        Lecce     Sassuolo     0     1  0.589232  0.410768   
232       23        Milan     Atalanta     2     0  0.299975  0.700025   
233       23  Salernitana        Monza     3     0  0.361873  0.638127   
234       23      Bologna        Inter     1     0  0.356339  0.643661   
235       23      Udinese       Spezia     2     2  0.555997  0.444003   
236       23       Verona   Fiorentina     0     3  0.646464  0.353536   
237       23        Lazio    Sampdoria     1     0  0.682945  0.317055   
238       23    Cremonese         Roma     2     1  0.634322  0.365678   
239       23     Juventus       Torino     4     2  0.744959  0.255041   
240       24       Napoli        Lazio     0     1  0.393206  0.606794   
241       24        Monza       Empoli     2     1  0.568464  0.431536   
242       24     Atalanta      Udinese     0     0  0.402933  0.597067   
243       24   Fiorentina        Milan     2     1  0.453248  0.546752   
244       24         Roma     Juventus     1     0  0.797706  0.202294   
245       24       Spezia       Verona     0     0  0.607295  0.392705   
246       24    Sampdoria  Salernitana     0     0  0.521363  0.478637   
247       24        Inter        Lecce     2     0  0.491638  0.508362   
248       24       Torino      Bologna     1     0  0.533076  0.466924   
249       24     Sassuolo    Cremonese     3     2  0.499184  0.500816   
250       25       Spezia        Inter     2     1  0.443506  0.556494   
251       25       Empoli      Udinese     0     1  0.578214  0.421786   
252       25       Napoli     Atalanta     2     0  0.306201  0.693799   
253       25      Bologna        Lazio     0     0  0.452825  0.547175   
254       25        Lecce       Torino     0     2  0.678836  0.321164   
255       25    Cremonese   Fiorentina     0     2  0.582282  0.417718   
256       25       Verona        Monza     1     1  0.528680  0.471320   
257       25         Roma     Sassuolo     3     4  0.656745  0.343255   
258       25     Juventus    Sampdoria     4     2  0.806945  0.193055   
259       25        Milan  Salernitana     1     1  0.288680  0.711320   
260       26     Atalanta       Empoli     2     1  0.498451  0.501549   
261       26     Sassuolo       Spezia     1     0  0.555088  0.444912   
262       26  Salernitana      Bologna     2     2  0.337957  0.662043   
263       26      Udinese        Milan     3     1  0.372718  0.627282   
264       26        Monza    Cremonese     1     1  0.481463  0.518537   
265       26       Torino       Napoli     0     4  0.441815  0.558185   
266       26        Lazio         Roma     1     0  0.659979  0.340021   
267       26    Sampdoria       Verona     3     1  0.712719  0.287281   
268       26   Fiorentina        Lecce     1     0  0.678422  0.321578   
269       26        Inter     Juventus     0     1  0.541108  0.458892   

     over_true  over_pred  
230      False       True  
231      False      False  
232      False       True  
233       True       True  
234      False       True  
235       True      False  
236       True      False  
237      False      False  
238       True      False  
239       True      False  
240      False       True  
241       True      False  
242      False       True  
243       True       True  
244      False      False  
245      False      False  
246      False      False  
247      False       True  
248      False      False  
249       True       True  
250       True       True  
251      False      False  
252      False       True  
253      False       True  
254      False      False  
255      False      False  
256      False      False  
257       True      False  
258       True      False  
259      False       True  
260       True       True  
261      False      False  
262       True       True  
263       True    

In [71]:
Z = T[T[["under", "over"]].max(axis=1) > 0.7]

In [70]:
precision_recall_fscore_support(T["over_true"], T["over_pred"])

(array([0.63636364, 0.44444444]),
 array([0.58333333, 0.5       ]),
 array([0.60869565, 0.47058824]),
 array([24, 16]))

In [73]:
precision_recall_fscore_support(Z["over_true"], Z["over_pred"])

(array([0.25, 0.  ]),
 array([0.33333333, 0.        ]),
 array([0.28571429, 0.        ]),
 array([3, 3]))

In [15]:
1/w.predict_1x2_single_match("Napoli", "Milan")

pred
1    1.454842
2    7.478230
X    5.589125
Name: p, dtype: float64

In [31]:
1/(1 - (1/1.75 + 1/3.65))

6.468354430379745

In [ ]:
#cProfile.run('w.fit(df, n_iter=4)')

In [ ]:
X = load_prediction_dataset(league="seriea", season="22_23")

In [ ]:
preds = w.predict(X, how="under_over")

In [ ]:
X.join(preds)